# FINM3422 Coding Project 2: OTC Derivatives Pricing & Hedging

This notebook implements an object-oriented solution to price four OTC derivative trades and calculate portfolio-level hedging parameters. All data and valuations are as of close of market on **16 May 2025**. (update this text, add anything else? Introduce task)

## Imports and Setup

In [4]:
from Option_Classes import EuropeanOption
from Option_Classes import AmericanPutOption
from Option_Classes import UpAndInCallOption
from Option_Classes import BasketCallOption
from Hedging_Parameters import HedgingCalculator
from market_data import get_close_price
from market_data import get_volatility
from market_data import get_correlation_matrix
from market_data import get_all_market_data
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import importlib
from simple_discount_curve import get_discount_rate
from simple_discount_curve import get_discount_factor

## Market Data Inputs

In [5]:
tickers = ["BHP.AX", "CBA.AX", "WES.AX", "CSL.AX", "WDS.AX", "MQG.AX"]
market_data = get_all_market_data(tickers, date="2025-05-16", window=60)

spot = market_data["spot"]
vol = market_data["vol"]
corr = market_data["corr"]

## broken asf atm

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


TypeError: get_volatility() got multiple values for argument 'end_date'

## Discount Curve Construction

do we build discount curve in the .ipynb file, or can we just produce the discount rates in the .py file? fix with the new discount curve file

In [ ]:

# This module builds a discount curve from bank bills and bonds, and provides functions to retrieve discount factors and zero rates
from junk.instrument_classes import Bank_bill, Bond, Portfolio
from curve_classes_and_functions import YieldCurve

# Builds bootstrapped discount curve from bank bills and bonds. 

def build_discount_curve():
    # Step 1: Create instruments. Data pulled from Bloomberg. 
    bill1 = Bank_bill(face_value=100, maturity=0.25, price=98.5)
    bill2 = Bank_bill(face_value=100, maturity=0.5, price=97.3)
    bill1.set_cash_flows()
    bill2.set_cash_flows()

    bond1 = Bond(face_value=100, maturity=1.0, coupon=0.04, frequency=4, price=99.2)
    bond2 = Bond(face_value=100, maturity=2.0, coupon=0.045, frequency=4, price=98.7)
    bond3 = Bond(face_value=100, maturity=3.0, coupon=0.05, frequency=4, price=97.0)
    bond1.set_cash_flows()
    bond2.set_cash_flows()
    bond3.set_cash_flows()

    # Step 2: Add to portfolio
    portfolio = Portfolio()
    portfolio.add_bank_bill(bill1)
    portfolio.add_bank_bill(bill2)
    portfolio.add_bond(bond1)
    portfolio.add_bond(bond2)
    portfolio.add_bond(bond3)
    portfolio.set_cash_flows()

    # Step 3: Bootstrap yield curve
    yc = YieldCurve()
    yc.set_constituent_portfolio(portfolio)
    yc.bootstrap()

    return yc

# Retrieves the discount factor for a given maturity from the bootstrapped yield curve
def get_discount_factor(yield_curve, maturity):
    return yield_curve.get_discount_factor(maturity)

# Returns the interpolated zero rate for a given maturity from the yield curve
def get_zero_rate(yield_curve, maturity):
    return yield_curve.get_zero_rate(maturity)


## Trade 1: BHP European Call

description of the trade

### Market Data

In [41]:
# Extract relevant data for option pricing and hedging
ticker = "BHP.AX"
date = "2025-05-16"

spot_bhp = get_close_price(ticker, date) #good
vol_bhp = get_volatility(ticker, end_date=date, window=60) #good
strike_bhp = 0.98 * spot_bhp

def time_to_expiry(start, end):
    return (end - start).days / 365.0

expiry_bhp = time_to_expiry(datetime(2025, 5, 16), datetime(2027, 9, 15)) # 2.33 Years



print(f"Spot Price: {spot_bhp}, Volatility: {vol_bhp}, Strike Price: {strike_bhp}, Expiry: {expiry_bhp} years")


## remove the extra shit and make scalars, also build so that it extracts only the correlation matrix?

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Spot Price: 39.720001220703125, Volatility: 0.29673778004012413, Strike Price: 38.925601196289065, Expiry: 2.334246575342466 years


### Discount Rate

In [ ]:
# Given the time to expiry, find the discount factor and zero rate
rate_bhp = get_discount_rate(expiry_bhp)  # good
print(rate_bhp)


0.05592461897265064


### Valuation

In [43]:
# Instantiate and price the option
bhp_option = EuropeanOption(ticker=ticker,spot=spot_bhp, strike=strike_bhp, expiry=expiry_bhp, rate=rate_bhp, vol=vol_bhp, option_type="call")

print(f"Option Price: {bhp_option.price()}")


Option Price: 9.75947958252221


### Visualisation

### Hedging Parameters

In [45]:
# Use HedgingCalculator to get delta, gamma, vega, theta, rho

hc = HedgingCalculator()
greeks_bhp = hc.get_all_greeks(bhp_option, method="fd")

print("Greeks for BHP Option:")
for greek, value in greeks_bhp.items():
    print(f"{greek}: {value}")





Greeks for BHP Option:
delta: 0.7119820658759934
gamma: 0.01894765336629689
vega: 20.704382251783393
theta: 2.352667822942479
rho: 43.2312951431868


### Hedging Strategy

## Trade 2: CBA American Put

description of the trade

### Market Data

In [46]:
ticker_cba = "CBA.AX"
date = "2025-05-16"

spot_cba = get_close_price(ticker_cba, date) #good
vol_cba = get_volatility(ticker_cba, end_date=date, window=60) #good
strike_cba = 170.00

def time_to_expiry(start, end):
    return (end - start).days / 365.0

expiry_cba = time_to_expiry(datetime(2025, 5, 16), datetime(2026, 5, 15)) # 1 Year


print(f"Spot Price: {spot_cba}, Volatility: {vol_cba}, Strike Price: {strike_cba}, Expiry: {expiry_cba} years")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Spot Price: 169.66000366210938, Volatility: 0.26622126099171345, Strike Price: 170.0, Expiry: 0.9972602739726028 years


### Discount Rate

In [47]:
# Instantiate and price the option
rate_cba = get_discount_rate(expiry_cba)  # good
print(f"Rate for CBA Option: {rate_cba}")

Rate for CBA Option: 0.04920549734846802


### Valuation

In [48]:
# Instantiate and price the option
cba_option = AmericanPutOption(ticker=ticker_cba, spot=spot_cba, strike=strike_cba, expiry=expiry_cba, rate=rate_cba, vol=vol_cba, option_type="put", steps=100)
print(f"Option Price: {cba_option.price()}")

Option Price: 14.75904004063862


### Visualisation

### Hedging Parameters

In [49]:
# Use HedgingCalculator to get delta, gamma, vega, theta, rho
hc = HedgingCalculator()
greeks_cba = hc.get_all_greeks(cba_option, method="fd")

print("Greeks for CBA Option:")
for greek, value in greeks_cba.items():
    print(f"{greek}: {value}")

Greeks for CBA Option:
delta: -0.4383388487205764
gamma: 3.552713678800501e-11
vega: 64.27162179080375
theta: 5.763751711450759
rho: -57.267451487659216


### Hedging Strategy

## Trade 3: WES Barrier Call

description of the trade

### Market Data

In [50]:
ticker_wes = "WES.AX"
date = "2025-05-16"

spot_wes = get_close_price(ticker_wes, date) #good
vol_wes = get_volatility(ticker_wes, end_date=date, window=60) #good
strike_wes = 80
barrier_wes = 100

def time_to_expiry(start, end):
    return (end - start).days / 365.0

expiry_wes = time_to_expiry(datetime(2025, 5, 16), datetime(2027, 9, 15)) # 2.33 Years

print(f"Spot Price: {spot_wes}, Volatility: {vol_wes}, Strike Price: {strike_wes}, Expiry: {expiry_wes} years, Barrier: {barrier_wes}")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Spot Price: 82.55999755859375, Volatility: 0.21930636472137627, Strike Price: 80, Expiry: 2.334246575342466 years, Barrier: 100


### Discount Rate

In [51]:
rate_wes = get_discount_rate(expiry_wes) 
print(f"Rate for WES Option: {rate_wes}")

Rate for WES Option: 0.05592461897265064


### Valuation

In [52]:
# Instantiate and price the option
wes_option = UpAndInCallOption(ticker=ticker_wes, spot=spot_wes, strike=strike_wes, expiry=expiry_wes, rate=rate_wes, vol=vol_wes, barrier=barrier_wes, simulations=10000,steps=252)
print(f"Option Price: {wes_option.price()}")


Option Price: 16.755162109025306


### Visualisation

### Hedging Parameters

In [54]:
# Use HedgingCalculator to get delta, gamma, vega, theta, rho
hc = HedgingCalculator()
greeks_wes = hc.get_all_greeks(wes_option, method="fd")

print("Greeks for WES Option:")
for greek, value in greeks_wes.items():
    print(f"{greek}: {value}")

Greeks for WES Option:
delta: 8.152602397938935
gamma: 9028.002068442973
vega: 50.047780201412984
theta: 124.95893871636082
rho: -50.77610915671826


### Hedging Strategy

## Trade 4: Basket Call

description of the trade

### Market Data

In [55]:
# Basket Call Option Inputs
tickers_basket = ["BHP.AX", "CSL.AX", "WDS.AX", "MQG.AX"]
date = "2025-05-16"
spot = {ticker: get_close_price(ticker, date) for ticker in tickers_basket}
vol = {ticker: get_volatility(ticker, end_date=date, window=60) for ticker in tickers_basket}
spot_basket = [spot[t] for t in tickers_basket]
vol_basket = [vol[t] for t in tickers_basket]
basket_corr = get_correlation_matrix(tickers_basket, window=60, end_date=date)
weights = [0.10, 0.35, 0.15, 0.40]
strike_basket = 175.0

from datetime import datetime
expiry_basket = time_to_expiry(datetime(2025, 5, 16), datetime(2025, 7, 17))

print("Spot prices:", spot_basket)
print("Volatilities:", vol_basket)
print(f"Strike: {strike_basket}, Expiry: {expiry_basket:.2f} yrs")
print(basket_corr)


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Spot prices: [39.720001220703125, 241.82000732421875, 21.920000076293945, 207.35000610351562]
Volatilities: [0.29673778004012413, 0.2645795058608444, 0.40561362662594136, 0.3774390594028384]
Strike: 175.0, Expiry: 0.17 yrs
Ticker    BHP.AX    CSL.AX    MQG.AX    WDS.AX
Ticker                                        
BHP.AX  1.000000  0.451883  0.444790  0.549149
CSL.AX  0.451883  1.000000  0.450133  0.407607
MQG.AX  0.444790  0.450133  1.000000  0.647139
WDS.AX  0.549149  0.407607  0.647139  1.000000


### Discount Rate

In [19]:
import importlib
import simple_discount_curve
importlib.reload(simple_discount_curve)
from simple_discount_curve import get_discount_rate, get_discount_factor

rate_basket = get_discount_rate(expiry_basket)
print(f"Discount rate: {rate_basket:.4%}")

Discount rate: 8.8975%


### Valuation

In [60]:
# Instantiate and price the option
basket_option = BasketCallOption(tickers=tickers_basket, spot_prices=spot_basket, weights=weights, strike=strike_basket, expiry=expiry_basket, rate=rate_basket, vol=vol_basket, corr_matrix=basket_corr, simulations=10000)
print(f"Basket Option Price: {basket_option.price()}")


Basket Option Price: 9.136523134441145


### Hedging Parameters

In [62]:
# Use HedgingCalculator to get delta, gamma, vega, theta, rho
hc = HedgingCalculator()
greeks_basket = hc.get_all_greeks(basket_option, method="fd")

print("Greeks for Basket Option:")
for greek, value in greeks_basket.items():
    print(f"{greek}: {value}")

Greeks for Basket Option:
delta: [np.float64(-5.239013872917564), np.float64(3.2317989915751077), np.float64(-0.31027731986723595), np.float64(0.6093577405529516)]
gamma: [np.float64(-3698.1662105306686), np.float64(-6400.1723464519955), np.float64(-361.62464123702165), np.float64(921.1266710041243)]
vega: [np.float64(-9.470850651493734), np.float64(5.2754250327903485), np.float64(-13.68718064580694), np.float64(10.199610373659063)]
theta: 84.93626392159403
rho: 203.4170758319931


### Hedging Strategy

## Portfolio Valuation and Hedging Summary

In [ ]:
# Collect all option prices and Greek exposures into summary DataFrame
# Sum portfolio value and Greek sensitivities

## Design and Model Justification

- Why Black-Scholes, Binomial Tree, and Monte Carlo were chosen
- Why finite difference for Greeks
- Why the bump sizes used
- Accuracy vs flexibility trade-offs

## Conclusion

- Modular, OOP-driven pricing system
- Accurate pricing and hedging of diverse OTC trades
- Integration of real market data and theory